In [1]:
import json
import utils.molecule_tools as mol
import numpy as np
import json
from collections import Counter
import utils.qonversion_tools as qonvert
import utils.bit_tools as bit
import utils.linalg_tools as la
import itertools
from matplotlib import pyplot as plt
# OpenFermion libraries
from openfermion import MolecularData
from openfermionpyscf import run_pyscf
from openfermion.ops import FermionOperator, QubitOperator
from openfermion.transforms import jordan_wigner, bravyi_kitaev
from openfermion.transforms import get_fermion_operator
from openfermion.circuits import (uccsd_singlet_get_packed_amplitudes,
                               uccsd_singlet_generator)
from pyscf import gto, scf, cc

Will store matrices in sparse form


/home/tweaving/.local/lib/python3.8/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')


In [2]:
with open('data/molecule_data.json', 'r') as data:
    molecule_data = json.load(data)

In [3]:
atoms, coords, multiplicity, charge, basis, sym_sector = molecule_data['Be_STO-3G_SINGLET'].values()

In [4]:
moltap=mol.construct_molecule(atoms, coords, charge, multiplicity, basis, ucc_threshold=1e-3, taper=True, sym_sector=sym_sector)
moluntap=mol.construct_molecule(atoms, coords, charge, multiplicity, basis, ucc_threshold=1e-3)

Attempting to taper 10 --> 5 qubits
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Energies match in sector (1, 1, 1, 1, 1), tapering successful!



In [7]:
moltap['uccsdansatz']

{'YIIXX': (1.734723475976807e-18-0.4999999999999999j),
 'XIIYX': (-1.734723475976807e-18+0.4999999999999999j),
 'XIIXY': (-5.204170427930421e-18+0.49999999999999994j),
 'YIIYY': (-5.204170427930421e-18+0.49999999999999994j),
 'IYIXX': (1.734723475976807e-18-0.4999999999999999j),
 'IXIYX': (-1.734723475976807e-18+0.4999999999999999j),
 'IXIXY': (-5.204170427930421e-18+0.49999999999999994j),
 'IYIYY': (-5.204170427930421e-18+0.49999999999999994j),
 'IIYXX': (-1.734723475976807e-18-0.4999999999999999j),
 'IIXYX': (1.734723475976807e-18+0.4999999999999999j),
 'IIXXY': (-5.204170427930421e-18+0.49999999999999994j),
 'IIYYY': (-5.204170427930421e-18+0.49999999999999994j)}

In [6]:
mult = Counter(atoms)
speciesname = ''
for a in mult.keys():
    speciesname += (a+str(mult[a])+'_')
speciesname += str(basis)

# ensure correct format for geometry
geometry = []
for index, a in enumerate(atoms):
    geometry.append((a, coords[index]))

# construct with PySCF through OpenFermion for actual molecular calculations (including coupled cluster amps)
moldata = MolecularData(geometry, basis, multiplicity, charge, description=speciesname)
#delete_input = True
#delete_output = True
molecule = run_pyscf(moldata,
                      run_scf=True,
                      run_mp2=True,
                      run_cisd=True,
                      run_ccsd=True,
                      run_fci=True)
num_electrons =molecule.n_electrons # total number of particles
num_particles =(molecule.get_n_alpha_electrons(),
                molecule.get_n_beta_electrons()) # number of alpha (spin up) and beta (spin down) electrons
num_qubits = 2*molecule.n_orbitals
# construct second quantised Hamiltonian:
ham_2ndQ = get_fermion_operator(molecule.get_molecular_hamiltonian())
# construct second quantised UCCSD operator:
ccsd_single_amps = molecule.ccsd_single_amps
ccsd_double_amps = molecule.ccsd_double_amps
packed_amps = uccsd_singlet_get_packed_amplitudes(ccsd_single_amps,  ccsd_double_amps, num_qubits, num_electrons)
ucc_2ndQ = uccsd_singlet_generator(packed_amps, num_qubits, num_electrons)

hf_config= ''.join(['1' for i in range(num_electrons)]+['0' for i in range(num_qubits-num_electrons)])
print(hf_config)

111100000000


In [7]:
ucc_2ndQ

-0.00030795433637807485 [0^ 4] +
0.0019024109678974534 [0^ 4 1^ 5] +
0.0009064336270445089 [0^ 6 1^ 7] +
0.0009064336270445094 [0^ 8 1^ 9] +
0.0004742387082844445 [0^ 10] +
0.0004862598310061071 [0^ 10 1^ 11] +
0.00040798151647315235 [0^ 10 2^ 4] +
0.00040798151647315235 [0^ 10 3^ 5] +
-0.00030795433637807485 [1^ 5] +
0.0019024109678974534 [1^ 5 0^ 4] +
0.0009064336270445089 [1^ 7 0^ 6] +
0.0009064336270445094 [1^ 9 0^ 8] +
0.0004742387082844445 [1^ 11] +
0.0004862598310061071 [1^ 11 0^ 10] +
0.00040798151647315235 [1^ 11 2^ 4] +
0.00040798151647315235 [1^ 11 3^ 5] +
-0.037484399331219824 [2^ 4] +
0.017141621708289423 [2^ 4 3^ 5] +
0.014017635470433993 [2^ 6 3^ 7] +
0.014017635470433991 [2^ 8 3^ 9] +
-0.0033503033732066915 [2^ 10] +
-0.00040798151647315235 [2^ 10 0^ 4] +
-0.00040798151647315235 [2^ 10 1^ 5] +
0.05577395168389476 [2^ 10 3^ 11] +
-0.037484399331219824 [3^ 5] +
0.017141621708289423 [3^ 5 2^ 4] +
0.014017635470433993 [3^ 7 2^ 6] +
0.014017635470433991 [3^ 9 2^ 8] +
-0.0033

In [8]:
ham_mat = qonvert.dict_to_WeightedPauliOperator(qonvert.QubitOperator_to_dict(jordan_wigner(ham_2ndQ), num_qubits)).to_matrix()
gsnrg, gsvec = la.get_ground_state(ham_mat)
print(gsnrg)
gsprobs = [abs(amp)**2 for amp in gsvec]
bstates = [bit.int_to_bin(i, num_qubits) for i in range(2**num_qubits)]

-7.882762230972004


In [9]:
sig_state_order = [s for s in sorted(list(zip(gsprobs, bstates)), key=lambda x:-x[0]) if s[0]>1e-3]
sig_state_order

[(0.9759572276560825, '111100000000'),
 (0.012144188720822343, '110000000011'),
 (0.0032300081299143015, '110001000010'),
 (0.003230008129914104, '110010000001'),
 (0.0013748904008232057, '111001000000'),
 (0.0013748904008226354, '110110000000'),
 (0.0010596284109832639, '110011000000')]

In [10]:
sig_ucc = FermionOperator()

for amp, state in sig_state_order:
    exCite = [index for index, b in enumerate(zip(state, hf_config)) if len(set(b))==2]
    if len(exCite)==2:
        sig_ucc += FermionOperator([(exCite[0],1),(exCite[1],0)])
        sig_ucc -= FermionOperator([(exCite[1],1),(exCite[0],0)])
    if len(exCite)==4:
        order1 = [2, 0, 3, 1]
        order2 = [3, 1, 2, 0]
        sig_ucc += FermionOperator([(cite, (index+1)%2) for index, cite in enumerate([exCite[i] for i in order1])])
        sig_ucc -= FermionOperator([(cite, (index+1)%2) for index, cite in enumerate([exCite[i] for i in order1][::-1])])
        sig_ucc += FermionOperator([(cite, (index+1)%2) for index, cite in enumerate([exCite[i] for i in order2])])
        sig_ucc -= FermionOperator([(cite, (index+1)%2) for index, cite in enumerate([exCite[i] for i in order2][::-1])])

sig_ucc

1.0 [2^ 4] +
-1.0 [2^ 4 3^ 5] +
-1.0 [2^ 4 3^ 11] +
-1.0 [2^ 5 3^ 10] +
-1.0 [2^ 10 3^ 11] +
1.0 [3^ 5] +
-1.0 [3^ 5 2^ 4] +
-1.0 [3^ 10 2^ 5] +
-1.0 [3^ 11 2^ 4] +
-1.0 [3^ 11 2^ 10] +
-1.0 [4^ 2] +
1.0 [4^ 2 5^ 3] +
1.0 [4^ 2 11^ 3] +
1.0 [5^ 2 10^ 3] +
-1.0 [5^ 3] +
1.0 [5^ 3 4^ 2] +
1.0 [10^ 2 11^ 3] +
1.0 [10^ 3 5^ 2] +
1.0 [11^ 3 4^ 2] +
1.0 [11^ 3 10^ 2]

In [11]:
reduccsd

NameError: name 'reduccsd' is not defined